In [1]:
import os
import math
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from rdkit import Chem

import torch
from torch_geometric.data import Data

# ========== 路径配置 ==========
DATA_PATH = Path("/root/fusion_dataset/Invertebrates_EC50_unique.xlsx")  # 换成你实际的数据集

GRAPH_OUT_DIR = Path("/root/Invertebrates_EC50_multi_fusion/graph/graph_outputs")
GRAPH_OUT_DIR.mkdir(parents=True, exist_ok=True)

GRAPH_PT_PATH      = GRAPH_OUT_DIR / "Invertebrates_EC50_graphs.pt"   # 图对象列表
GRAPH_ROWID_PATH   = GRAPH_OUT_DIR / "row_id_graph.npy"       # 每个图对应 df 的 row_id
GRAPH_EMB_PATH     = GRAPH_OUT_DIR / "reg_graph_embeddings.npy"
GRAPH_ROWID_EMB    = GRAPH_OUT_DIR / "row_id_graph_for_emb.npy"

print("图数据将保存到:", GRAPH_PT_PATH)
print("row_id 将保存到:", GRAPH_ROWID_PATH)
print("图嵌入将保存到:", GRAPH_EMB_PATH)

# ========== 读 Excel ==========
df = pd.read_excel(DATA_PATH, engine="openpyxl")

# 确保有 row_id
if "row_id" not in df.columns:
    df = df.reset_index().rename(columns={"index": "row_id"})
df["row_id"] = df["row_id"].astype(int)

# 标签：若没有 mgperL_log 就自己算一个（和文本端保持统一）
if "mgperL_log" not in df.columns:
    df["mgperL_log"] = np.log10(df["mgperL"].astype(float))

required_cols = [
    "row_id",
    "SMILES_Canonical_RDKit",
    "mgperL_log",
    "Duration_Value(hour)",
    "Effect",
    "Endpoint",
]
for c in required_cols:
    assert c in df.columns, f"df 缺少列: {c}"

print("数据行数:", len(df))
print(df[required_cols].head())


图数据将保存到: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs/Invertebrates_EC50_graphs.pt
row_id 将保存到: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs/row_id_graph.npy
图嵌入将保存到: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs/reg_graph_embeddings.npy
数据行数: 3620
   row_id    SMILES_Canonical_RDKit  mgperL_log  Duration_Value(hour) Effect  \
0       0        [Cl-].[Cl-].[Zn+2]    0.113943                  96.0    ITX   
1       1  O=S(=O)([O-])[O-].[Zn+2]    0.397940                  24.0    ITX   
2       2        [Cl-].[Cl-].[Pb+2]    1.610660                  96.0    ITX   
3       3  O=S(=O)([O-])[O-].[Cu+2]    0.278754                  24.0    ITX   
4       4  O=S(=O)([O-])[O-].[Cu+2]   -0.221849                  96.0    ITX   

  Endpoint  
0     EC50  
1     EC50  
2     EC50  
3     EC50  
4     EC50  


In [2]:
from rdkit import Chem
from rdkit.Chem import rdchem

# ========= 一些辅助配置 =========
ATOM_LIST = ["C","N","O","F","Cl","Br","I","S","P","B","Si","other"]
HYB_LIST  = [
    rdchem.HybridizationType.SP,
    rdchem.HybridizationType.SP2,
    rdchem.HybridizationType.SP3,
    rdchem.HybridizationType.SP3D,
    rdchem.HybridizationType.SP3D2,
    "other"
]

def one_hot(x, allowed):
    return [int(x == a) for a in allowed]

def atom_to_feature(atom: Chem.Atom):
    """
    节点特征：
    - 元素 one-hot
    - 原子度数 degree (0-5, 6+)
    - 形式电荷 formal charge (-2~2)
    - 杂化态 hybridization
    - 总氢数 (0-4, 5+)
    - 是否在环中
    - 是否芳香
    - 手性标签（可以视为可选）
    """
    symbol = atom.GetSymbol()
    if symbol not in ATOM_LIST:
        symbol = "other"

    hyb = atom.GetHybridization()
    if hyb not in HYB_LIST:
        hyb = "other"

    # 元素 one-hot
    feat_symbol = one_hot(symbol, ATOM_LIST)

    # 度数（0-5, >=6）
    deg = atom.GetDegree()
    deg_list = [0,1,2,3,4,5,"6+"]
    deg_clamp = deg if deg <= 5 else "6+"
    feat_degree = one_hot(deg_clamp, deg_list)

    # 形式电荷（-2, -1, 0, 1, 2）
    fc = atom.GetFormalCharge()
    fc_list = [-2,-1,0,1,2]
    fc_clamp = fc if fc in fc_list else 0
    feat_charge = one_hot(fc_clamp, fc_list)

    # 杂化态
    feat_hyb = one_hot(hyb, HYB_LIST)

    # 氢原子个数（0-4, >=5）
    num_h = atom.GetTotalNumHs()
    h_list = [0,1,2,3,4,"5+"]
    h_clamp = num_h if num_h <= 4 else "5+"
    feat_num_h = one_hot(h_clamp, h_list)

    # 在环 / 芳香性
    feat_ring      = [int(atom.IsInRing())]
    feat_aromatic  = [int(atom.GetIsAromatic())]

    # 手性信息（可选）
    chiral_tag = atom.GetChiralTag()
    chiral_list = [
        rdchem.ChiralType.CHI_UNSPECIFIED,
        rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
        rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
        "other"
    ]
    if chiral_tag not in chiral_list:
        chiral_tag = "other"
    feat_chiral = one_hot(chiral_tag, chiral_list)

    feat = (
        feat_symbol +
        feat_degree +
        feat_charge +
        feat_hyb +
        feat_num_h +
        feat_ring +
        feat_aromatic +
        feat_chiral
    )
    return feat


def bond_to_feature(bond: Chem.Bond):
    """
    边特征：
    - bond type one-hot (single/double/triple/other)
    - 是否共轭
    - 是否在环中
    （stereo 想加也可以再扩一组 one-hot）
    """
    bt = bond.GetBondType()
    if bt == rdchem.BondType.SINGLE:
        bt_vec = [1,0,0,0]
    elif bt == rdchem.BondType.DOUBLE:
        bt_vec = [0,1,0,0]
    elif bt == rdchem.BondType.TRIPLE:
        bt_vec = [0,0,1,0]
    else:
        bt_vec = [0,0,0,1]  # aromatic or others

    conj    = [int(bond.GetIsConjugated())]
    in_ring = [int(bond.IsInRing())]

    return bt_vec + conj + in_ring   # 总维度 = 4 + 1 + 1 = 6


def mol_to_graph(smiles, y, row_id):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # ------- 节点特征 -------
    atom_feats = []
    for atom in mol.GetAtoms():
        atom_feats.append(atom_to_feature(atom))
    x = torch.tensor(atom_feats, dtype=torch.float32)

    # ------- 边特征 -------
    edge_index_list = []
    edge_attr_list  = []

    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        bfeat = bond_to_feature(bond)

        # 无向图：i->j, j->i
        edge_index_list.append([i, j])
        edge_index_list.append([j, i])
        edge_attr_list.append(bfeat)
        edge_attr_list.append(bfeat)

    if len(edge_index_list) == 0:
        # 没键的奇怪分子，直接跳过
        return None

    edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()  # (2, E)
    edge_attr  = torch.tensor(edge_attr_list, dtype=torch.float32)                 # (E, edge_dim)

    y_tensor      = torch.tensor([y], dtype=torch.float32)
    row_id_tensor = torch.tensor([row_id], dtype=torch.long)

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=y_tensor,
        row_id=row_id_tensor,
    )
    return data

# ===== 批量构图（这段和原来的 Cell 1 一样，可以保留）=====
graph_list = []
valid_row_ids = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    smiles = row["SMILES_Canonical_RDKit"]
    y      = float(row["mgperL_log"])
    rid    = int(row["row_id"])

    g = mol_to_graph(smiles, y, rid)
    if g is None:
        continue

    graph_list.append(g)
    valid_row_ids.append(rid)

print(f"成功构建图样本数: {len(graph_list)}")

row_id_graph = np.array(valid_row_ids, dtype=np.int64)

torch.save(graph_list, GRAPH_PT_PATH)
np.save(GRAPH_ROWID_PATH, row_id_graph)

print("✅ graph_list 已保存:", GRAPH_PT_PATH)
print("✅ row_id_graph 已保存:", GRAPH_ROWID_PATH)


  0%|          | 0/3620 [00:00<?, ?it/s][00:26:27] SMILES Parse Error: syntax error while parsing: CCCC[Sn](|[S]CC(=O)OCC(CC)CCCC)(|[S]CC(=O)OCC(CC)CCCC)CCCC
[00:26:27] SMILES Parse Error: check for mistakes around position 10:
[00:26:27] CCCC[Sn](|[S]CC(=O)OCC(CC)CCCC)(|[S]CC(=O
[00:26:27] ~~~~~~~~~^
[00:26:27] SMILES Parse Error: Failed parsing SMILES 'CCCC[Sn](|[S]CC(=O)OCC(CC)CCCC)(|[S]CC(=O)OCC(CC)CCCC)CCCC' for input: 'CCCC[Sn](|[S]CC(=O)OCC(CC)CCCC)(|[S]CC(=O)OCC(CC)CCCC)CCCC'
[00:26:27] SMILES Parse Error: syntax error while parsing: [Cl]|[Sn](|[Cl])(|[Cl])CCCC
[00:26:27] SMILES Parse Error: check for mistakes around position 5:
[00:26:27] [Cl]|[Sn](|[Cl])(|[Cl])CCCC
[00:26:27] ~~~~^
[00:26:27] SMILES Parse Error: Failed parsing SMILES '[Cl]|[Sn](|[Cl])(|[Cl])CCCC' for input: '[Cl]|[Sn](|[Cl])(|[Cl])CCCC'
[00:26:27] SMILES Parse Error: syntax error while parsing: CCCCCCCCCCCC[S]|[Sn](|[S]CCCCCCCCCCCC)(CCCC)CCCC
[00:26:27] SMILES Parse Error: check for mistakes around position 1

成功构建图样本数: 3213
✅ graph_list 已保存: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs/Invertebrates_EC50_graphs.pt
✅ row_id_graph 已保存: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs/row_id_graph.npy


In [3]:
from sklearn.model_selection import GroupShuffleSplit

# 重新读一遍 graph_list 和 row_id_graph（保证从磁盘加载）
graph_list = torch.load(GRAPH_PT_PATH)
row_id_graph = np.load(GRAPH_ROWID_PATH)

print("图样本数:", len(graph_list))

# 按 row_id 回到 df，拿到 SMILES 作为 group
df_indexed = df.set_index("row_id")
smiles_all_graph = df_indexed.loc[row_id_graph, "SMILES_Canonical_RDKit"].values

print("smiles_all_graph 长度:", len(smiles_all_graph))

groups = smiles_all_graph  # 每个图的 group 标签就是对应 SMILES

gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_idx_gnn, val_idx_gnn = next(gss.split(np.zeros(len(row_id_graph)), groups=groups))

train_idx_gnn = np.array(train_idx_gnn, dtype=np.int64)
val_idx_gnn   = np.array(val_idx_gnn, dtype=np.int64)

print("GNN train 样本数:", len(train_idx_gnn))
print("GNN val   样本数:", len(val_idx_gnn))


/tmp/ipykernel_1760/4169944597.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_list = torch.load(GRAPH_PT_PATH)


图样本数: 3213
smiles_all_graph 长度: 3213
GNN train 样本数: 2573
GNN val   样本数: 640


In [4]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv, global_mean_pool

class GraphDataset(Dataset):
    def __init__(self, data_list, indices):
        self.data_list = data_list
        self.indices = np.array(indices, dtype=np.int64)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        return self.data_list[self.indices[idx]]

class GATv2Regressor(nn.Module):
    def __init__(self, in_channels, hidden_channels=256, num_layers=2, heads=4, edge_dim=None, dropout=0.05):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)

        self.convs = nn.ModuleList()

        assert heads == 1 or hidden_channels % heads == 0, "hidden_channels 必须能被 heads 整除"

        out_channels = hidden_channels // heads if heads > 1 else hidden_channels

        # 第一层
        self.convs.append(
            GATv2Conv(
                in_channels,
                out_channels,
                heads=heads,
                concat=(heads > 1),
                dropout=dropout,
                edge_dim=edge_dim,
            )
        )

        # 后续层
        for _ in range(num_layers - 1):
            self.convs.append(
                GATv2Conv(
                    hidden_channels,
                    out_channels,
                    heads=heads,
                    concat=(heads > 1),
                    dropout=dropout,
                    edge_dim=edge_dim,
                )
            )

        # 图级 pooling 之后接一个回归头
        self.reg_head = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_channels, 1),
        )

    def encode_graph(self, data):
        x, edge_index = data.x, data.edge_index
        edge_attr = getattr(data, "edge_attr", None)
        batch = data.batch

        for conv in self.convs:
            x = conv(x, edge_index, edge_attr)
            x = F.elu(x)
            x = self.dropout(x)

        # 图级 embedding
        graph_emb = global_mean_pool(x, batch)  # (B, hidden_channels)
        return graph_emb

    def forward(self, data):
        graph_emb = self.encode_graph(data)
        out = self.reg_head(graph_emb).squeeze(-1)
        return out


In [5]:
from sklearn.metrics import r2_score, mean_absolute_error
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备:", device)

def compute_metrics(y_true, y_pred):
    return r2_score(y_true, y_pred), mean_absolute_error(y_true, y_pred)

train_graph_dataset = GraphDataset(graph_list, train_idx_gnn)
val_graph_dataset   = GraphDataset(graph_list, val_idx_gnn)

ENC_BATCH_SIZE_G = 64
MAX_EPOCHS_G     = 100
PATIENCE_G       = 10
LR_G             = 3e-3
WEIGHT_DECAY_G   = 1e-4

train_loader = DataLoader(
    train_graph_dataset,
    batch_size=ENC_BATCH_SIZE_G,
    shuffle=True,
    drop_last=False,
)
val_loader = DataLoader(
    val_graph_dataset,
    batch_size=ENC_BATCH_SIZE_G,
    shuffle=False,
    drop_last=False,
)

# 节点/边特征维度
node_feat_dim = graph_list[0].x.size(-1)
edge_feat_dim = graph_list[0].edge_attr.size(-1)

gnn_model = GATv2Regressor(
    in_channels=node_feat_dim,
    hidden_channels=256,
    num_layers=3,
    heads=4,
    edge_dim=edge_feat_dim,
    dropout=0.05,
).to(device)

optimizer = AdamW(gnn_model.parameters(), lr=LR_G, weight_decay=WEIGHT_DECAY_G)

steps_per_epoch = max(1, math.ceil(len(train_loader)))
scheduler = OneCycleLR(
    optimizer,
    max_lr=LR_G,
    epochs=MAX_EPOCHS_G,
    steps_per_epoch=steps_per_epoch,
)

best_val_loss = float("inf")
best_state_dict = None
best_epoch_g = 0
patience_counter = 0

for epoch in range(1, MAX_EPOCHS_G + 1):
    # ---- 训练 ----
    gnn_model.train()
    train_losses = []
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = gnn_model(batch)
        y_true = batch.y.view(-1).to(device)
        loss = torch.nn.functional.l1_loss(pred, y_true)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.item())

    avg_train_loss = float(np.mean(train_losses)) if train_losses else float("nan")

    # ---- 验证 ----
    gnn_model.eval()
    val_losses = []
    val_true, val_pred = [], []
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            pred = gnn_model(batch)
            y_true = batch.y.view(-1).to(device)
            loss = torch.nn.functional.l1_loss(pred, y_true)
            val_losses.append(loss.item())
            val_true.append(y_true.cpu().numpy())
            val_pred.append(pred.cpu().numpy())

    avg_val_loss = float(np.mean(val_losses)) if val_losses else float("nan")
    val_true = np.concatenate(val_true, axis=0)
    val_pred = np.concatenate(val_pred, axis=0)
    val_r2, val_mae = compute_metrics(val_true, val_pred)

    print(
        f"[GNN] Epoch {epoch}/{MAX_EPOCHS_G} | "
        f"TrainLoss={avg_train_loss:.4f} | "
        f"ValLoss={avg_val_loss:.4f} | "
        f"Val_R2={val_r2:.4f} | Val_MAE={val_mae:.4f}"
    )

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch_g = epoch
        best_state_dict = {k: v.cpu().clone() for k, v in gnn_model.state_dict().items()}
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE_G:
            print(f"Early stopping at epoch {epoch} (no improvement in {PATIENCE_G} epochs).")
            break

# 恢复最佳 checkpoint
if best_state_dict is not None:
    gnn_model.load_state_dict(best_state_dict)
    gnn_model.to(device)
    print(f"\n✅ GNN Encoder restored to best checkpoint (Epoch={best_epoch_g}, ValLoss={best_val_loss:.4f}).")
else:
    print("\n⚠️ 未记录到更优 checkpoint，当前权重即为最终权重。")


使用设备: cuda
[GNN] Epoch 1/100 | TrainLoss=0.9977 | ValLoss=0.9309 | Val_R2=-0.1131 | Val_MAE=0.9309
[GNN] Epoch 2/100 | TrainLoss=0.9219 | ValLoss=0.8946 | Val_R2=-0.0115 | Val_MAE=0.8946
[GNN] Epoch 3/100 | TrainLoss=0.9122 | ValLoss=0.8907 | Val_R2=-0.0192 | Val_MAE=0.8907
[GNN] Epoch 4/100 | TrainLoss=0.9083 | ValLoss=0.8935 | Val_R2=-0.0611 | Val_MAE=0.8935
[GNN] Epoch 5/100 | TrainLoss=0.9004 | ValLoss=0.8673 | Val_R2=0.0428 | Val_MAE=0.8673
[GNN] Epoch 6/100 | TrainLoss=0.8866 | ValLoss=0.8377 | Val_R2=0.0815 | Val_MAE=0.8377
[GNN] Epoch 7/100 | TrainLoss=0.8721 | ValLoss=0.8330 | Val_R2=-0.0085 | Val_MAE=0.8330
[GNN] Epoch 8/100 | TrainLoss=0.8317 | ValLoss=0.8093 | Val_R2=0.1251 | Val_MAE=0.8093
[GNN] Epoch 9/100 | TrainLoss=0.8143 | ValLoss=0.7838 | Val_R2=0.0689 | Val_MAE=0.7838
[GNN] Epoch 10/100 | TrainLoss=0.8077 | ValLoss=0.8028 | Val_R2=-0.0028 | Val_MAE=0.8028
[GNN] Epoch 11/100 | TrainLoss=0.7995 | ValLoss=0.7967 | Val_R2=0.0412 | Val_MAE=0.7967
[GNN] Epoch 12/100 | Tra

In [6]:
from torch_geometric.loader import DataLoader

gnn_model.eval()

all_embeds = []
all_row_ids_for_emb = []

full_loader = DataLoader(
    graph_list,
    batch_size=ENC_BATCH_SIZE_G,
    shuffle=False,
    drop_last=False,
)

with torch.no_grad():
    for batch in full_loader:
        batch = batch.to(device)
        graph_emb = gnn_model.encode_graph(batch)  # (B, hidden_dim)
        all_embeds.append(graph_emb.cpu().numpy())
        all_row_ids_for_emb.append(batch.row_id.view(-1).cpu().numpy())

emb_all = np.concatenate(all_embeds, axis=0)               # (N_graph, d_g)
row_id_graph_for_emb = np.concatenate(all_row_ids_for_emb, axis=0).astype(np.int64)

print("emb_all shape:", emb_all.shape)
print("row_id_graph_for_emb shape:", row_id_graph_for_emb.shape)

np.save(GRAPH_EMB_PATH, emb_all)
np.save(GRAPH_ROWID_EMB, row_id_graph_for_emb)

print("✅ 图嵌入与对应 row_id 已保存。")


emb_all shape: (3213, 256)
row_id_graph_for_emb shape: (3213,)
✅ 图嵌入与对应 row_id 已保存。


In [7]:
import numpy as np
import pandas as pd
from pathlib import Path

# 载入刚刚保存的嵌入
emb_all_g = np.load(GRAPH_EMB_PATH)              # (N_graph, d_g)
row_id_graph = np.load(GRAPH_ROWID_EMB)          # (N_graph,)

print("emb_all_g shape:", emb_all_g.shape)
print("row_id_graph shape:", row_id_graph.shape)

df_indexed = df.set_index("row_id")

# 确保这些 row_id 都在 df 中
assert set(row_id_graph).issubset(set(df_indexed.index)), "有 row_id 不在 df 中，请检查预处理。"

meta_df = df_indexed.loc[row_id_graph, ["Duration_Value(hour)", "Effect", "Endpoint"]].copy()

# ===== 这里改了：直接用原始 Duration，不做 log =====
meta_df["Duration_raw"] = meta_df["Duration_Value(hour)"].astype(float)

# 分类特征 one-hot
effect_dum   = pd.get_dummies(meta_df["Effect"], prefix="eff")
endpoint_dum = pd.get_dummies(meta_df["Endpoint"], prefix="ep")

# 用 Duration_raw + one-hot(effect) + one-hot(endpoint)
meta_feat_df = pd.concat(
    [meta_df[["Duration_raw"]], effect_dum, endpoint_dum],
    axis=1
)

X_meta = meta_feat_df.values.astype(np.float32)
X_graph = emb_all_g.astype(np.float32)

X_all_graph = np.concatenate([X_graph, X_meta], axis=1)

y_all_graph = df_indexed.loc[row_id_graph, "mgperL_log"].values.astype(np.float32)
smiles_all_graph = df_indexed.loc[row_id_graph, "SMILES_Canonical_RDKit"].values

print("X_all_graph shape:", X_all_graph.shape)
print("y_all_graph shape:", y_all_graph.shape)
print("样本数一致吗？", X_all_graph.shape[0] == y_all_graph.shape[0] == len(smiles_all_graph))


emb_all_g shape: (3213, 256)
row_id_graph shape: (3213,)
X_all_graph shape: (3213, 260)
y_all_graph shape: (3213,)
样本数一致吗？ True


In [8]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr, randint
import numpy as np
import json
import pickle

# ===== 1) SMILES 分组的 8:2 划分（RF 自己一套，与 GNN 划分独立）=====
groups = smiles_all_graph  # group 标签就是 SMILES

gss_rf = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=2025)
train_idx_rf, test_idx_rf = next(gss_rf.split(X_all_graph, y_all_graph, groups=groups))

train_idx_rf = np.array(train_idx_rf, dtype=np.int64)
test_idx_rf  = np.array(test_idx_rf, dtype=np.int64)

X_train_rf = X_all_graph[train_idx_rf]
X_test_rf  = X_all_graph[test_idx_rf]
y_train_rf = y_all_graph[train_idx_rf]
y_test_rf  = y_all_graph[test_idx_rf]
groups_train_rf = groups[train_idx_rf]

print("RF train 样本数:", X_train_rf.shape[0])
print("RF test  样本数:", X_test_rf.shape[0])

# ===== 2) 在 train 集上做十折 GroupKFold + RandomizedSearchCV 找超参 =====
base_rf = RandomForestRegressor(
    n_jobs=-1,
    random_state=42,
)

param_distributions = {
    "n_estimators": randint(200, 1001),        # [200, 1000]
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.3, 0.5],
}

# 固定一套十折划分，后面 OOF 也用同一套
cv_inner = GroupKFold(n_splits=10)
cv_indices_graph = list(cv_inner.split(X_train_rf, y_train_rf, groups_train_rf))

rf_search = RandomizedSearchCV(
    estimator=base_rf,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="r2",
    cv=cv_indices_graph,   # 用固定 folds
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

print("\n==== 开始在 RF(train) 上做十折随机搜索 (GroupKFold) ====")
rf_search.fit(X_train_rf, y_train_rf, groups=groups_train_rf)

best_params_rf = rf_search.best_params_
best_cv_score  = rf_search.best_score_

print("\n===== RF 最优超参（train 上 10-fold GroupKFold）=====")
print(best_params_rf)
print(f"CV 平均 R^2: {best_cv_score:.4f}")


RF train 样本数: 2581
RF test  样本数: 632

==== 开始在 RF(train) 上做十折随机搜索 (GroupKFold) ====
Fitting 10 folds for each of 30 candidates, totalling 300 fits
[CV] END max_depth=20, max_features=0.5, min_samples_leaf=1, min_samples_split=10, n_estimators=306; total time= 1.3min
[CV] END max_depth=10, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=585; total time= 2.4min
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=760; total time= 1.2min
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=760; total time= 1.2min
[CV] END max_depth=20, max_features=0.3, min_samples_leaf=4, min_samples_split=5, n_estimators=675; total time= 4.8min
[CV] END max_depth=10, max_features=0.3, min_samples_leaf=4, min_samples_split=2, n_estimators=330; total time= 3.4min
[CV] END max_depth=None, max_features=0.3, min_samples_leaf=4, min_samples_split=2, n_estimators=366; total time= 3.8min
[CV] END max_de

In [9]:
# ===== 一些小工具 =====
def regression_metrics(y_true, y_pred):
    return {
        "r2": r2_score(y_true, y_pred),
        "mae": mean_absolute_error(y_true, y_pred),
        "rmse": np.sqrt(mean_squared_error(y_true, y_pred)),
        "r": pearsonr(y_true, y_pred)[0],
    }

# ===== 3) 用同一套 10 折 + 最优超参，算 train OOF 预测 =====
print("\n==== 基于最优超参，在同一套 10 折上计算 train OOF 预测 ====")

oof_pred_train_rf = np.zeros_like(y_train_rf, dtype=float)

for fold_idx, (tr_idx, val_idx) in enumerate(cv_indices_graph, 1):
    print(f"  -> OOF fold {fold_idx} / {len(cv_indices_graph)}")
    rf_fold = RandomForestRegressor(
        **best_params_rf,
        n_jobs=-1,
        random_state=42 + fold_idx,
    )
    rf_fold.fit(X_train_rf[tr_idx], y_train_rf[tr_idx])
    oof_pred_train_rf[val_idx] = rf_fold.predict(X_train_rf[val_idx])

metrics_oof = regression_metrics(y_train_rf, oof_pred_train_rf)

print("\n===== RF 图像端：train OOF 表现 =====")
for k, v in metrics_oof.items():
    print(f"{k}: {v:.4f}")

# ===== 4) 用最优超参在整个 train80% 上拟合最终 RF =====
best_rf = RandomForestRegressor(
    **best_params_rf,
    n_jobs=-1,
    random_state=42,
)
best_rf.fit(X_train_rf, y_train_rf)

y_pred_train = best_rf.predict(X_train_rf)
y_pred_test  = best_rf.predict(X_test_rf)

metrics_train = regression_metrics(y_train_rf, y_pred_train)
metrics_test  = regression_metrics(y_test_rf, y_pred_test)

print("\n===== RF 训练集表现 =====")
for k, v in metrics_train.items():
    print(f"{k}: {v:.4f}")

print("\n===== RF 测试集表现（独立 20% SMILES 组）=====")
for k, v in metrics_test.items():
    print(f"{k}: {v:.4f}")

# ===== 5) 保存模型 & 指标 & 划分索引 & OOF / 预测（用于后期融合）=====
RF_MODEL_PATH   = GRAPH_OUT_DIR / "rf_graph_model.pkl"
RF_METRICS_PATH = GRAPH_OUT_DIR / "rf_graph_metrics.json"
RF_PARAMS_PATH  = GRAPH_OUT_DIR / "rf_graph_best_params.json"
RF_SPLIT_PATH   = GRAPH_OUT_DIR / "rf_graph_train_test_idx.npz"

with open(RF_MODEL_PATH, "wb") as f:
    pickle.dump(best_rf, f)

# 指标里加上 oof 一块存
with open(RF_METRICS_PATH, "w") as f:
    json.dump(
        {
            "train": metrics_train,
            "test":  metrics_test,
            "oof":   metrics_oof,
            "cv_best_r2": float(best_cv_score),
        },
        f,
        indent=2
    )

with open(RF_PARAMS_PATH, "w") as f:
    json.dump(best_params_rf, f, indent=2)

# 保存 8:2 的索引（原来就有）
np.savez(
    RF_SPLIT_PATH,
    train_idx=train_idx_rf,
    test_idx=test_idx_rf,
)

# 🔹新增：给后期融合用的关键文件（命名跟文本端保持风格一致）
np.save(GRAPH_OUT_DIR / "rf_graph_y_train.npy",        y_train_rf.astype(np.float32))
np.save(GRAPH_OUT_DIR / "rf_graph_y_test.npy",         y_test_rf.astype(np.float32))
np.save(GRAPH_OUT_DIR / "rf_graph_oof_pred_train.npy", oof_pred_train_rf.astype(np.float32))
np.save(GRAPH_OUT_DIR / "rf_graph_y_pred_test.npy",    y_pred_test.astype(np.float32))
np.save(GRAPH_OUT_DIR / "rf_graph_train_idx.npy",      train_idx_rf.astype(np.int64))
np.save(GRAPH_OUT_DIR / "rf_graph_test_idx.npy",       test_idx_rf.astype(np.int64))

print("\n✅ RF 图像端：模型、指标、OOF、预测和索引已保存到:", GRAPH_OUT_DIR)
print("   - rf_graph_oof_pred_train.npy  (train OOF，用于 late fusion)")
print("   - rf_graph_y_pred_test.npy     (test 预测，用于 late fusion)")
print("   - rf_graph_train_idx.npy / rf_graph_test_idx.npy (与文本端对齐的 8:2 划分)")



==== 基于最优超参，在同一套 10 折上计算 train OOF 预测 ====
  -> OOF fold 1 / 10
  -> OOF fold 2 / 10
  -> OOF fold 3 / 10
  -> OOF fold 4 / 10
  -> OOF fold 5 / 10
  -> OOF fold 6 / 10
  -> OOF fold 7 / 10
  -> OOF fold 8 / 10
  -> OOF fold 9 / 10
  -> OOF fold 10 / 10

===== RF 图像端：train OOF 表现 =====
r2: 0.4946
mae: 0.6001
rmse: 0.8184
r: 0.7040

===== RF 训练集表现 =====
r2: 0.8871
mae: 0.2670
rmse: 0.3869
r: 0.9498

===== RF 测试集表现（独立 20% SMILES 组）=====
r2: 0.5074
mae: 0.6145
rmse: 0.8497
r: 0.7138

✅ RF 图像端：模型、指标、OOF、预测和索引已保存到: /root/Invertebrates_EC50_multi_fusion/graph/graph_outputs
   - rf_graph_oof_pred_train.npy  (train OOF，用于 late fusion)
   - rf_graph_y_pred_test.npy     (test 预测，用于 late fusion)
   - rf_graph_train_idx.npy / rf_graph_test_idx.npy (与文本端对齐的 8:2 划分)
